# Cleaning: Dirty Cafe Sales Data

This notebook focuses exclusively on data preparation and cleaning. The input is a dataset containing transaction data from a café, which shows signs of contamination (missing values, inconsistent formats, text errors).

### 📂 Dataset Info
* **Source:** [Kaggle - Dirty Cafe Sales Dataset](https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training)
* **File:** `dirty_cafe_sales.csv`
* **Description:** Synthetic dirty data created specifically for cleaning practice.

## Loading data and libraries

In [169]:
# Libraries
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("dirty_cafe_sales.csv")

# Number of rows and columns
print(f"Loaded Dataset: {df.shape[0]} rows, {df.shape[1]} columns")

Loaded Dataset: 10000 rows, 8 columns


## Basic data inspection

In [170]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [171]:
# Printing names of columns
print(df.columns.tolist())

# Renaming columns for consistency
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Checking the correctness of names
print(df.columns.tolist())

['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent', 'Payment Method', 'Location', 'Transaction Date']
['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent', 'payment_method', 'location', 'transaction_date']


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10000 non-null  object
 1   item              9667 non-null   object
 2   quantity          9862 non-null   object
 3   price_per_unit    9821 non-null   object
 4   total_spent       9827 non-null   object
 5   payment_method    7421 non-null   object
 6   location          6735 non-null   object
 7   transaction_date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


🚩 **Initial Findings**
Based on the `df.info()` output, immediate cleaning is required for data types:

* **Numeric Columns (`Quantity`, `Price Per Unit`, `Total Spent`):** Currently stored as `object` (string) instead of numbers, likely due to non-numeric values.

* **Date Column (`Transaction Date`):** Currently stored as `object`. Needs conversion to `datetime` format for time-based analysis.

### Duplicates and missing values

#### Duplicates

In [173]:
print(f"Number of duplicated rows: {df.duplicated().sum()}")

Number of duplicated rows: 0


#### Missing values

In [174]:
print(f"Number of missing values: \n {df.isna().sum()}")

Number of missing values: 
 transaction_id         0
item                 333
quantity             138
price_per_unit       179
total_spent          173
payment_method      2579
location            3265
transaction_date     159
dtype: int64


📉 Missing Values Summary

The output above reveals significant gaps in the dataset that require immediate attention:

* **High Severity:** The `Location` (3265 missing) and `Payment Method` (2579 missing) columns are heavily compromised, missing approximately **25-33%** of data. Dropping these rows would result in massive data loss.

* **Moderate Severity:** Essential operational columns like `Item`, `Price Per Unit`, and `Quantity` are missing hundreds of values. Since these are required for calculating total sales, we cannot simply ignore them.

> **Strategy Required:** Simple removal (`dropna`) is not a viable option for columns like `Location` as we would lose too much data. We will need to combine **imputation** (filling gaps with "Unknown" or statistical averages) with **row removal** for critical missing operational data.

## Data Cleaning 


### `Quantity`

In [ ]:
# Standardize text format
df['quantity'] = df['quantity'].astype(str).str.strip().str.title()

# Force convert to numeric
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')

# Handle Missing Values 
# Assumption: A transaction implies at least 1 item was sold.
df['quantity'] = df['quantity'].fillna(1)

# Convert to Integer
df['quantity'] = df['quantity'].astype(int)

# 5. Verification
print(f"Data type: {df['quantity'].dtype}")
print("-" * 30)
print(df['quantity'].describe())

Data type: int32
------------------------------
count    10000.00000
mean         2.93130
std          1.45079
min          1.00000
25%          2.00000
50%          3.00000
75%          4.00000
max          5.00000
Name: quantity, dtype: float64


### `Price` and `Total Spent` 
Both `Price Per Unit` and `Total Spent` columns share identical issues with the `Quantity` column: mixed types, string errors ("ERROR", "UNKNOWN"), and missing values.

**Approach:**
To adhere to the **DRY (Don't Repeat Yourself)** principle, we will define a reusable helper function `clean_float_column`.

**Logic:**
1.  **Coerce to Numeric:** Convert column to numbers; non-numeric strings become `NaN`.
2.  **Imputation:** Fill missing values (`NaN`) with the **median** of the column. Using the median is robust against outliers (unlike the mean).
3.  **Return:** A clean column of floats.

In [ ]:
# Defining Cleaning Function

def clean_float_column(series):
    """
    Cleans a pandas Series by converting to numeric and filling NaNs with the median.
    """
    # Force convert to numeric (errors -> NaN)
    cleaned_series = pd.to_numeric(series, errors='coerce')
    
    # Calculate Median (ignoring NaNs automatically)
    median_val = cleaned_series.median()
    
    # Fill Missing Values
    cleaned_series = cleaned_series.fillna(median_val)
    
    return cleaned_series

# Applying Function

# Apply to 'Price Per Unit'
df['price_per_unit'] = clean_float_column(df['price_per_unit'])

# Apply to 'Total Spent'
df['total_spent'] = clean_float_column(df['total_spent'])

# Verification
print("Data Types after cleaning:")
print(df[['price_per_unit', 'total_spent']].dtypes)
print("-" * 30)
print("Preview of cleaned values:")
print(df[['price_per_unit', 'total_spent']].head())

Data Types after cleaning:
price_per_unit    float64
total_spent       float64
dtype: object
------------------------------
Preview of cleaned values:
   price_per_unit  total_spent
0             2.0          4.0
1             3.0         12.0
2             1.0          8.0
3             5.0         10.0
4             2.0          4.0


### `Item`, `Payment Method` & `Location`

All three categorical columns (`Item`, `Payment Method`, `Location`) share identical issues: they contain inconsistencies like "UNKNOWN", "ERROR", missing values, and mixed casing.

**Strategy:**
We will define a reusable function `clean_categorical_column` that:
1.  **Standardizes Text:** Converts everything to Title Case (e.g., "latte" -> "Latte", "UNKNOWN" -> "Unknown").
2.  **Unifies Missing Data:** Maps all error placeholders ("Error", "Nan", empty strings) to a single category `'Unknown'`.

This ensures consistency across all categorical variables in the dataset.

In [180]:
# Defining Categorical Cleaning Function

def clean_categorical_column(series):
    """
    Cleans a categorical column by standardizing text case 
    and replacing error values with 'Unknown'.
    """
    # Convert to string and standardise format 
    # This handles "UNKNOWN" -> "Unknown", "nan" -> "Nan", "latte" -> "Latte"
    cleaned_series = series.astype(str).str.strip().str.title()
    
    # Define bad values to map to 'Unknown'
    error_values = ['Error', 'Nan', '', 'Unknown'] 
    
    # Replace bad values
    cleaned_series = cleaned_series.replace(error_values, 'Unknown')
    
    return cleaned_series

# Applying Function
df['item'] = clean_categorical_column(df['item'])
df['payment_method'] = clean_categorical_column(df['payment_method'])
df['location'] = clean_categorical_column(df['location'])

# Verification
print("Unique Items:", df['item'].unique().tolist())
print("-" * 30)
print("Unique Payment Methods:", df['payment_method'].unique().tolist())
print("-" * 30)
print("Unique Locations:", df['location'].unique().tolist())

Unique Items: ['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'Unknown', 'Sandwich', 'Juice', 'Tea']
------------------------------
Unique Payment Methods: ['Credit Card', 'Cash', 'Unknown', 'Digital Wallet']
------------------------------
Unique Locations: ['Takeaway', 'In-Store', 'Unknown']


### Transaction date
The `Transaction Date` column contains invalid values like "ERROR" and "UNKNOWN", along with missing data. Since the date is a factual record critical for time-series analysis, it cannot be reliably imputed.

**Strategy:**
* **Method:** Drop rows.
* **Justification:** Dropping approx. 1.6% of data is preferable to fabricating timestamps, which would distort sales trends and daily volume analysis.

**Action:**
1.  Convert the column to datetime objects using `errors='coerce'` (turns invalid text into `NaT`).
2.  Remove all rows where the date is `NaT`.

In [182]:
# Convert to Datetime
# 'errors='coerce'' handles "ERROR", "UNKNOWN" by turning them into NaT (Not a Time)
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

# Assess Data Loss
n_missing_dates = df['transaction_date'].isna().sum()
print(f"Invalid dates to be dropped: {n_missing_dates} rows ({n_missing_dates/len(df):.2%})")

# Drop rows
df = df.dropna(subset=['transaction_date'])

# Verification
print("-" * 30)
print(f"Final dataset shape: {df.shape}")
print(f"Date column type: {df['transaction_date'].dtype}")

Invalid dates to be dropped: 460 rows (4.60%)
------------------------------
Final dataset shape: (9540, 8)
Date column type: datetime64[ns]


## Final Validation & Export

### Validation

In [185]:
# Check Data Types & Non-Null Counts
# We expect: no 'object' types for numbers/dates, and 0 missing values.
print("Data Info: \n")
df.info()

print("-" * 30)

# Verify No Missing Values
# Should be all zeros.
print("Remaining Missing Values: \n")
print(df.isna().sum())

print("-" * 30)

Data Info: 

<class 'pandas.core.frame.DataFrame'>
Index: 9540 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    9540 non-null   object        
 1   item              9540 non-null   object        
 2   quantity          9540 non-null   int32         
 3   price_per_unit    9540 non-null   float64       
 4   total_spent       9540 non-null   float64       
 5   payment_method    9540 non-null   object        
 6   location          9540 non-null   object        
 7   transaction_date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int32(1), object(4)
memory usage: 633.5+ KB
------------------------------
Remaining Missing Values: 

transaction_id      0
item                0
quantity            0
price_per_unit      0
total_spent         0
payment_method      0
location            0
transaction_date    0
dtype: int64
---------------

In [186]:
# Statistical Summary
# Check for anomalies (e.g., min quantity < 1, negative prices)
print("Statistical Summary: \n")
print(df.describe())

Statistical Summary: 

          quantity  price_per_unit  total_spent               transaction_date
count  9540.000000     9540.000000  9540.000000                           9540
mean      2.928931        2.952673     8.868816  2023-07-01 23:00:31.698113536
min       1.000000        1.000000     1.000000            2023-01-01 00:00:00
25%       2.000000        2.000000     4.000000            2023-04-01 00:00:00
50%       3.000000        3.000000     8.000000            2023-07-02 00:00:00
75%       4.000000        4.000000    12.000000            2023-10-02 00:00:00
max       5.000000        5.000000    25.000000            2023-12-31 00:00:00
std       1.449786        1.243449     5.861817                            NaN


### Export

In [187]:
# Save to CSV
output_filename = 'clean_cafe_sales.csv'

df.to_csv(output_filename, index=False)

print(f"✅ Success! Cleaned data saved to '{output_filename}'")
print(f"Final dataset shape: {df.shape}")

✅ Success! Cleaned data saved to 'clean_cafe_sales.csv'
Final dataset shape: (9540, 8)
